In [1]:
import os
import sys
import numpy as np
import pandas as pd
import json

sys.path.append('../')
sys.path.append('../src/')
sys.path.append('../spell/')

import Reader
import ParamsExtractor3
import DataPreprocessor
import EncodeCommand
import DeepLearningAnomalyDetection2

2024-08-23 12:52:02.495918: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-23 12:52:02.544431: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-23 12:52:03.331763: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# First type of log file:
# laurel_anomalous_new
fp = '../data/laurel_anomalous_new/audit.log'
fps = [fp]
fps += [fp + f'.{str(i)}' for i in range(1,8)]
print(fps)

['../data/laurel_anomalous_new/audit.log', '../data/laurel_anomalous_new/audit.log.1', '../data/laurel_anomalous_new/audit.log.2', '../data/laurel_anomalous_new/audit.log.3', '../data/laurel_anomalous_new/audit.log.4', '../data/laurel_anomalous_new/audit.log.5', '../data/laurel_anomalous_new/audit.log.6', '../data/laurel_anomalous_new/audit.log.7']


In [3]:
fp = fps[0]

data_list = []
lines_limit = 1000 # limit reading lines, done for memory constraints
i = 0

with open(fp, 'r') as file:
    for line in file:
        i+=1
        if i > lines_limit:
            break
            # Find the index of the first '{'
        index = line.find('{')
        if index != -1:
            json_data = line[index:]
            data = json.loads(json_data)
            data_list.append(data)
                
df = pd.DataFrame(data_list)

In [4]:
df.head()

,ID,SYSCALL,CWD,PATH,PROCTITLE,EXECVE
0,1723373608.018:15989,"{'arch': '0xc000003e', 'syscall': 59, 'success...",{'cwd': '/home/matteo/Desktop/priv_esc_scripts...,"[{'item': 0, 'name': '/home/matteo/.cargo/bin/...","{'ARGV': ['find', '/', '-type', 'f', '-iname',...",NaN
1,1723373608.018:15990,"{'arch': '0xc000003e', 'syscall': 59, 'success...",{'cwd': '/home/matteo/Desktop/priv_esc_scripts...,"[{'item': 0, 'name': '/home/matteo/.local/bin/...","{'ARGV': ['find', '/', '-type', 'f', '-iname',...",NaN
2,1723373608.018:15991,"{'arch': '0xc000003e', 'syscall': 59, 'success...",{'cwd': '/home/matteo/Desktop/priv_esc_scripts...,"[{'item': 0, 'name': '/home/matteo/bin/ls', 'n...","{'ARGV': ['find', '/', '-type', 'f', '-iname',...",NaN
3,1723373608.018:15992,"{'arch': '0xc000003e', 'syscall': 59, 'success...",{'cwd': '/home/matteo/Desktop/priv_esc_scripts...,"[{'item': 0, 'name': '/usr/share/Modules/bin/l...","{'ARGV': ['find', '/', '-type', 'f', '-iname',...",NaN
4,1723373608.018:15993,"{'arch': '0xc000003e', 'syscall': 59, 'success...",{'cwd': '/home/matteo/Desktop/priv_esc_scripts...,"[{'item': 0, 'name': '/usr/local/bin/ls', 'nam...","{'ARGV': ['find', '/', '-type', 'f', '-iname',...",NaN


In [5]:
def normalize(df, col):
    if col not in df.columns:
        print(f"Error, column {col} not in df")
        return None
    df_expanded = pd.json_normalize(df[col])
    #print(df_expanded.head())
    df = df.drop(col, axis=1)
    df = df.join(df_expanded, rsuffix=f'_{col}')
    return df

In [6]:
df.columns

Index(['ID', 'SYSCALL', 'CWD', 'PATH', 'PROCTITLE', 'EXECVE'], dtype='object')

In [7]:
df = normalize(df, 'CWD')
df = normalize(df, 'PATH')
df = normalize(df, 'SYSCALL')
df = normalize(df, 'PROCTITLE')
df = normalize(df, 'EXECVE')
df = normalize(df, 0)
df = normalize(df, 1)
df.head()

df['ARGV_PROCTITLE_str'] = df['ARGV_PROCTITLE'].apply(lambda x: ' '.join(x))

df['unix_time'] = pd.to_numeric(df['ID'].str.split(':').str[0])
df['timedelta'] = df['unix_time'].diff()
df['time'] = pd.to_datetime(df['unix_time'], unit='s')

df['ppid_time'] = pd.to_datetime(pd.to_numeric(df['PPID.EVENT_ID'].str.split(':').str[0]), unit='s')
df['ppid_timedelta'] = df['ppid_time'].diff()

df['pid_time'] = pd.to_datetime(pd.to_numeric(df['PID.EVENT_ID'].str.split(':').str[0]), unit='s')
df['pid_timedelta'] = df['pid_time'].diff()

df = df.drop(['ID', 'PID.EVENT_ID', 'PPID.EVENT_ID', 'unix_time', 'ARGV_PROCTITLE', 'time', 'pid_time', 'ppid_time'], axis=1)

In [8]:
for col in df.columns:
    print(f'{col} : {df.iloc[1][col]}')

cwd : /home/matteo/Desktop/priv_esc_scripts/linpeas
arch : 0xc000003e
syscall : 59
success : no
exit : -2
items : 1
ppid : 28988
pid : 29144
auid : 1000
uid : 1000
gid : 1000
euid : 1000
suid : 1000
fsuid : 1000
egid : 1000
sgid : 1000
fsgid : 1000
tty : pts1
ses : 3
comm : find
exe : /usr/bin/find
subj : unconfined_u:unconfined_r:unconfined_t:s0-s0:c0.c1023
key : exec_log
ARCH : x86_64
SYSCALL : execve
AUID : matteo
UID : matteo
GID : matteo
EUID : matteo
SUID : matteo
FSUID : matteo
EGID : matteo
SGID : matteo
FSGID : matteo
ARGV : ['0x7ffc841ac9a0', '0x55861f767810', '0x7ffc841ad090', '0x7ffc841afa80']
UID_GROUPS : ['matteo', 'wheel', 'vboxsf']
PPID.comm : find
PPID.exe : /usr/bin/find
PPID.ppid : 3241
argc : nan
ARGV_EXECVE : nan
item : 0
name : /home/matteo/.local/bin/ls
nametype : UNKNOWN
cap_fp : 0x0
cap_fi : 0x0
cap_fe : 0
cap_fver : 0x0
cap_frootid : 0
inode : nan
dev : nan
mode : nan
ouid : nan
ogid : nan
rdev : nan
obj : nan
OUID : nan
OGID : nan
item_1 : nan
name_1 : nan
in

In [9]:
columns = ['auid', 'uid', 'gid', 'euid', 'suid', 'fsuid', 'egid', 'sgid', 'fsgid']

def check_row_id_num(row):
    return pd.Series({col: row[col] == 0 for col in columns})

checks = df.apply(check_row_id_num, axis=1)
df['num_id_anomalies'] = checks.sum(axis=1)
df = df.drop(columns, axis=1)

In [10]:
columns = ['AUID', 'UID', 'GID', 'EUID', 'SUID', 'FSUID', 'EGID', 'SGID', 'FSGID']

# Use apply to efficiently check values for each row
def check_id_row(row):    
    return pd.Series({col: int(not(row[col] in row['UID_GROUPS'])) for col in columns})

checks = df.apply(check_id_row, axis=1)

df['id_anomalies'] = checks.sum(axis=1)
df = df.drop(columns, axis=1)
df = df.drop('UID_GROUPS', axis=1)

In [11]:
for col in df.columns:
    print(f'{col} : {df.iloc[1][col]}')

cwd : /home/matteo/Desktop/priv_esc_scripts/linpeas
arch : 0xc000003e
syscall : 59
success : no
exit : -2
items : 1
ppid : 28988
pid : 29144
tty : pts1
ses : 3
comm : find
exe : /usr/bin/find
subj : unconfined_u:unconfined_r:unconfined_t:s0-s0:c0.c1023
key : exec_log
ARCH : x86_64
SYSCALL : execve
ARGV : ['0x7ffc841ac9a0', '0x55861f767810', '0x7ffc841ad090', '0x7ffc841afa80']
PPID.comm : find
PPID.exe : /usr/bin/find
PPID.ppid : 3241
argc : nan
ARGV_EXECVE : nan
item : 0
name : /home/matteo/.local/bin/ls
nametype : UNKNOWN
cap_fp : 0x0
cap_fi : 0x0
cap_fe : 0
cap_fver : 0x0
cap_frootid : 0
inode : nan
dev : nan
mode : nan
ouid : nan
ogid : nan
rdev : nan
obj : nan
OUID : nan
OGID : nan
item_1 : nan
name_1 : nan
inode_1 : nan
dev_1 : nan
mode_1 : nan
ouid_1 : nan
ogid_1 : nan
rdev_1 : nan
obj_1 : nan
nametype_1 : nan
cap_fp_1 : nan
cap_fi_1 : nan
cap_fe_1 : nan
cap_fver_1 : nan
cap_frootid_1 : nan
OUID_1 : nan
OGID_1 : nan
ARGV_PROCTITLE_str : find / -type f -iname .* ! -path /sys/* ! -

In [12]:
# filter final columns
columns_to_keep = ['cwd', 'exit', 'items', 'ppid', 'pid', 'comm', 'timedelta', 'pid_timedelta', 'ppid_timedelta', 'id_anomalies', 'num_id_anomalies', 'ARGV_PROCTITLE_str']
df = df [columns_to_keep]

In [13]:
df.iloc[1]

cwd                       /home/matteo/Desktop/priv_esc_scripts/linpeas
exit                                                                 -2
items                                                                 1
ppid                                                              28988
pid                                                               29144
comm                                                               find
timedelta                                                           0.0
pid_timedelta                                           0 days 00:00:00
ppid_timedelta                                          0 days 00:00:00
id_anomalies                                                          0
num_id_anomalies                                                      0
ARGV_PROCTITLE_str    find / -type f -iname .* ! -path /sys/* ! -pat...
Name: 1, dtype: object

In [14]:
os.getcwd()

'/home/rocky/repo/Thesis/notebooks_dataset_creation'

In [15]:
# Save this df to file
save_fp = '../data/laurel_anomalous_new/save1.csv'
df.to_csv(save_fp, index=False)

In [16]:
# Final Preprocessing Function
def preprocess_laurel(df):
    
    df = normalize(df, 'CWD')
    df = normalize(df, 'PATH')
    df = normalize(df, 'SYSCALL')
    df = normalize(df, 'PROCTITLE')
    df = normalize(df, 'EXECVE')
    df = normalize(df, 0)
    df = normalize(df, 1)
    print(df.head())

    columns_to_keep = ['cwd', 'exit', 'items', 'ppid', 'pid', 'comm', 'timedelta', 'pid_timedelta', 'ppid_timedelta', 'id_anomalies', 'num_id_anomalies', 'ARGV_PROCTITLE_str']
    columns_to_drop = ['ID', 'PID.EVENT_ID', 'PPID.EVENT_ID', 'unix_time', 'ARGV_PROCTITLE', 'time', 'pid_time', 'ppid_time']
    
    try:
        df['ARGV_PROCTITLE_str'] = df['ARGV_PROCTITLE'].apply(lambda x: ' '.join(x))
    except:
        columns_to_keep.remove('ARGV_PROCTITLE_str')
        pass
    
    df['unix_time'] = pd.to_numeric(df['ID'].str.split(':').str[0])
    df['timedelta'] = df['unix_time'].diff()
    df['time'] = pd.to_datetime(df['unix_time'], unit='s')

    try:
        df['ppid_time'] = pd.to_datetime(pd.to_numeric(df['PPID.EVENT_ID'].str.split(':').str[0]), unit='s')
        df['ppid_timedelta'] = df['ppid_time'].diff()
        columns_to_keep.remove('ppid_timedelta')
        columns_to_drop.remove('ppid_time')
    except:
        pass
    
    df['pid_time'] = pd.to_datetime(pd.to_numeric(df['PID.EVENT_ID'].str.split(':').str[0]), unit='s')
    df['pid_timedelta'] = df['pid_time'].diff()
    
    df = df.drop(columns_to_drop, axis=1)

    columns = ['auid', 'uid', 'gid', 'euid', 'suid', 'fsuid', 'egid', 'sgid', 'fsgid']
    
    def check_row_id_num(row):
        return pd.Series({col: row[col] == 0 for col in columns})
    
    checks = df.apply(check_row_id_num, axis=1)
    df['num_id_anomalies'] = checks.sum(axis=1)
    df = df.drop(columns, axis=1)

    columns = ['AUID', 'UID', 'GID', 'EUID', 'SUID', 'FSUID', 'EGID', 'SGID', 'FSGID']
    
    # Use apply to efficiently check values for each row
    def check_id_row(row):
        try:
            return pd.Series({col: int(not(row[col] in row['UID_GROUPS'])) for col in columns})
        except:
            pass
    
    checks = df.apply(check_id_row, axis=1)
    
    df['id_anomalies'] = checks.sum(axis=1)
    df = df.drop(columns, axis=1)
    df = df.drop('UID_GROUPS', axis=1)

    # filter final columns
    df = df[columns_to_keep]

    return df

In [17]:
def read(fp):
    data_list = []
    lines_limit = 1000 # limit reading lines, done for memory constraints
    i = 0
    
    with open(fp, 'r') as file:
        for line in file:
            i+=1
            if i > lines_limit:
                break
                # Find the index of the first '{'
            index = line.find('{')
            if index != -1:
                json_data = line[index:]
                data = json.loads(json_data)
                data_list.append(data)
                    
    df = pd.DataFrame(data_list)
    return df

In [18]:
def save(df, save_fp):
    # Save this df to file
    #save_fp = '../data/laurel_anomalous_new/save1.csv'
    df.to_csv(save_fp, index=False)

In [21]:
for i,fp in enumerate(fps):
    df = read(fp)
    print(df.head())
    df = preprocess_laurel(df)
    save(df, fp + '.csv')
    new_path = '../cleaned_data/anomalous' + str(i) + '.csv'
    save(df, new_path)

                     ID                                            SYSCALL  \
0  1723373608.018:15989  {'arch': '0xc000003e', 'syscall': 59, 'success...   
1  1723373608.018:15990  {'arch': '0xc000003e', 'syscall': 59, 'success...   
2  1723373608.018:15991  {'arch': '0xc000003e', 'syscall': 59, 'success...   
3  1723373608.018:15992  {'arch': '0xc000003e', 'syscall': 59, 'success...   
4  1723373608.018:15993  {'arch': '0xc000003e', 'syscall': 59, 'success...   

                                                 CWD  \
0  {'cwd': '/home/matteo/Desktop/priv_esc_scripts...   
1  {'cwd': '/home/matteo/Desktop/priv_esc_scripts...   
2  {'cwd': '/home/matteo/Desktop/priv_esc_scripts...   
3  {'cwd': '/home/matteo/Desktop/priv_esc_scripts...   
4  {'cwd': '/home/matteo/Desktop/priv_esc_scripts...   

                                                PATH  \
0  [{'item': 0, 'name': '/home/matteo/.cargo/bin/...   
1  [{'item': 0, 'name': '/home/matteo/.local/bin/...   
2  [{'item': 0, 'name': '/

KeyError: "['ppid_time'] not found in axis"

In [ ]:
fps

In [ ]:
anomalous_dataset_fp = fps[0] + '.csv'
new_path = '../cleaned_data/anomalous.csv'
save(df, new_path)

In [ ]:
df.head()